## Extract tiles from a multiplexed Thunder experiment

The goal of this script is to resave individual tiles from a lif file as tiffs to be compatible with assembly using ASHLAR algorithm.

The tiles will be saved in a format:

WELL_r_ROUND_ch_CHANNEL_TILE

Depending on the naming convention you may need to adjust name parsing to get the right images from the lif file and clean the string for saving.

In [2]:
import os
import json
from readlif.reader import LifFile
from tifffile import imwrite

from thunder_functions import create_tiles_99

In [1]:
# full path to the lif file
file_path = r'R:\Narayanan\202409_Bcell\20240716_Bcell.lif'

# path to the folder where the tiff files will be saved
dir_save = r''

# path to the file to store the dictionary
dict_path = r''

# expected size of a single frame
frame_size = 2048

In [3]:
# open lif image
image_stack = LifFile(file_path)

In [ ]:
# check the content of the lif file

img_list = [[image.name, image.dims] for image in image_stack.get_iter_image()]
print(len(img_list))
img_list

In [ ]:
# wells to be processed

well_list= list(set(['/'.join(image.name.split('/')[-2:]) for image in image_stack.get_iter_image() if not ('Merged' in image.name or '5X' in image.name)])) # get all wells
well_list = sorted(well_list, key=lambda x: (x.split('/')[0], int(x.split('/')[1])))
# well_list = []# example: ['D/3','D/9'] # specify manually
well_list

In [ ]:
# rounds to be processed

round_list = list(set([image.name.split('_')[0] for image in image_stack.get_iter_image() if not (('Merged' in image.name) or ('5X' in image.name))])) # get all wells
round_list = sorted(round_list, key=lambda x: int(x[1:]))
#round_list = []# example: ['R1','R2'] # specify manually
round_list

In [ ]:
for well in well_list:

    for r_int,r in enumerate(round_list):

        # print info about what is found
        img_list_names = [image.name for image in image_stack.get_iter_image() if ((r+'_' in image.name) and (well in image.name) and not('Merged' in image.name))]
        print(img_list_names)
        ch_list = [image.channels for image in image_stack.get_iter_image() if ((r+'_' in image.name) and (well in image.name) and not('Merged' in image.name))]
        print(ch_list)

        # get images
        img_list = [image for image in image_stack.get_iter_image() if ((r+'_' in image.name) and (well in image.name) and not('Merged' in image.name))]

        # extract tiles
        t_list = create_tiles_99(img_list, ch_list)

        # save tiles
        for ind,t in enumerate(t_list):

            for ch in range(sum(ch_list)):

                imwrite(os.path.join(dir_save,f"{well.replace('/','')}_r_{str(r[1:]).zfill(2)}_ch_{str(ch).zfill(2)}_{str(ind).zfill(2)}.tif"), t[ch,:,:].astype('uint16'))


### Generate experiment distionary

In [ ]:
my_dict = dict()

for well in well_list:

    for r_int,r in enumerate(round_list):

        # print info about what is found
        img_list_names = [image.name for image in image_stack.get_iter_image() if ((r+'_' in image.name) and (well in image.name) and not('Merged' in image.name))]
        print(img_list_names)
        ch_list = [image.channels for image in image_stack.get_iter_image() if ((r+'_' in image.name) and (well in image.name) and not('Merged' in image.name))]
        print(ch_list)

        channel_names = []
        for im_name in img_list_names:

            # get the channel names
            channel_round_names = im_name.split('/')[0].split('_')[1:]

            for ch in channel_round_names:
                ch = ch.split('-')[0] # remove this one to keep dyes names
                if ch == 'DAPI':
                    ch = f'DAPI{str(r_int).zfill(2)}'
                    

                channel_names.append(ch)
            
        for i,ch in enumerate(channel_names):
            print(f'{r_int}_ {i}: {ch}')
            my_dict[f'{r_int}_ {i}'] = ch

# save dictionary
with open(dict_path, 'w') as file:
     file.write(json.dumps(my_dict))